### _I, Library:_

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pytz
from time import sleep
import random


### _II, Scraping data:_

- The data is about the real estate for sale in my city, Hanoi

In [2]:
url = "https://alonhadat.com.vn/nha-dat/can-ban/dat-tho-cu-dat-o/1/ha-noi.html"

def scraping(url):
    """
    Scraping data.
    """
    result = requests.get(url)
    c = result.content
    soup = BeautifulSoup(c,"html.parser")
    lists = soup.find_all("div" , class_ = "content-item")
    df_list = []
    for e in lists:

        try:
            date = e.find("div", class_ = "ct_date").text
            if date == "Hôm nay":
                tz_VN = pytz.timezone('Asia/Bangkok')
                today = datetime.now(tz_VN)
                if today.month < 10:
                    date = str(today.day) + "/0" + str(today.month) + "/" + str(today.year)
                else:
                    date = str(today.day) + "/" + str(today.month) + "/" + str(today.year)
            elif date == "Hôm qua":
                tz_VN = pytz.timezone('Asia/Bangkok')
                today = datetime.now(tz_VN)
                if today.month < 10:
                    date = str(today.day-1) + "/0" + str(today.month) + "/" + str(today.year)
                else:
                    date = str(today.day-1) + "/" + str(today.month) + "/" + str(today.year)
        except:
            date = '-'
            
            
        try:
            road_width = e.find("span", class_ = "road-width").text
        except:
            road_width = "-"
        try:
            area = e.find("div", class_ = "ct_dt").text.split(": ")[1]
        except:
            area = "-"

        try:
            size = e.find("div", class_ = "ct_kt").text.split(": ")[1]
        except:
            size = "-"

        try:
            direction = e.find("div", class_ = "ct_direct").text.split(": ")[1]
        except:
            direction = "-"
        try:
            price = e.find("div", class_ = "ct_price").text.split(": ")[1]
        except:
            price = "-"
            
        try:
            location = e.find("div", class_  = "ct_dis").text
        except:
            price = "-"
        combine = [ date, road_width, area, size, direction, price, location]
        df_list.append(combine)
    
    return df_list
first_page = scraping(url)
first_page

[]

In [8]:
final_list = []
for numb in range(431,700):
    sleep(5)
    try:
        print(numb)
        url1 = "https://alonhadat.com.vn/nha-dat/can-ban/dat-tho-cu-dat-o/1/ha-noi/trang--"+str(numb)+".html"
        new = scraping(url1)
        final_list+=new
        if new==[]:
            break
    except:
        break

df = pd.DataFrame(final_list, columns=['Date', 'Road_width', 'Area', 'Size', 'Direction', 'Price', 'Location'])

df

431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482


,Date,Road_width,Area,Size,Direction,Price,Location
0,17/10/2023,7m,75 m2,---,_,"1,2 tỷ","Tỉnh lộ 420, Xã Bình Yên, Huyện Thạch Thất, Hà..."
1,17/10/2023,16m,169 m2,---,_,13 triệu / m2,"Quốc lộ 21A, Xã Phú Cát, Huyện Quốc Oai, Hà Nội"
2,17/10/2023,5m,86 m2,---,_,"1,2 tỷ","Quốc lộ 21A, Xã Phú Cát, Huyện Quốc Oai, Hà Nội"
3,17/10/2023,7m,110 m2,---,_,"1,5 tỷ","Tỉnh lộ 420, Xã Tân Xã, Huyện Thạch Thất, Hà Nội"
4,17/10/2023,4m,60 m2,---,_,600 triệu,"Tỉnh lộ 420, Xã Bình Yên, Huyện Thạch Thất, Hà..."
...,...,...,...,...,...,...,...
1015,16/10/2023,5m,320 m2,---,_,22 triệu / m2,"Xã Tiến Xuân, Huyện Thạch Thất, Hà Nội"
1016,16/10/2023,4m,1.029 m2,---,_,"3,2 tỷ","Xã Tiến Xuân, Huyện Thạch Thất, Hà Nội"
1017,16/10/2023,5m,468 m2,---,_,"2,7 tỷ","Xã Yên Bài, Huyện Ba Vì, Hà Nội"
1018,16/10/2023,5m,1.136 m2,---,_,"5,5 triệu / m2","Xã Tiến Xuân, Huyện Thạch Thất, Hà Nội"


In [9]:
# Update/create excel file

# Update: add new data to the existed excel file (if existed)
try:
    old_data = pd.read_excel("hnrealestate.xlsx")
    print(len(old_data))
    df_all = old_data.merge(df, on=["Date","Road_width","Area","Size","Direction","Price","Location"], how='outer')
    df_all.to_excel("hnrealestate.xlsx", sheet_name='Sheet1',index=False)

# Create a new one
except:
    df.to_excel("hnrealestate1.xlsx", sheet_name='Sheet1',index=False)

56813
